To Dos:
- Add sigma, phi and tau to optimization
  - How to draw from negative binomial with negative phi?
  - Tau as value to initialize y
  - why does sigma have a gradient of 0?
- Change loss function to negative log likelihood
- Toy Data
- Add prior knowledge to loss function
- Optimize for loops
- Check .clone() - Where do we need it and where is it redundant?



Haomann: Neg. Log Likelihood & Phi Variable
Anuar: Toy Data & Tau Variable
Timo: Sigma Variable

__This notebook is an approach proposed by Haoman to add NLL et Neg. Binomial distribution up on yt__

todos:
1. forward pass needs to be checked again for bugs
2. the model transformation and also the further steps like the likelihood lossfunction needs to be defined


In [1]:
import scipy.stats
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import expon, truncexpon, truncnorm, nbinom
import pandas as pd
import torch
from torch import nn
from torch import distributions

In [2]:
np.random.seed(seed=101)
torch.manual_seed(101)
torch.use_deterministic_algorithms(True)
dtype = torch.float
device = torch.device("cpu")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/MyDrive/CovidPrediction/

/content/drive/MyDrive/CovidPrediction


# Initialization

In [3]:
data = pd.read_csv('data/covid19model.csv')
data.head(3)

,date,hospit,serial_interval,delay_distr
0,2020-02-17,0,0.046535,0.013006
1,2020-02-18,0,0.087065,0.030046
2,2020-02-19,0,0.112061,0.044674


In [4]:
cero = torch.tensor(0, requires_grad=False, device=device, dtype=dtype)
num_impute = 6
observed_daily_hospit = torch.tensor(data.hospit, requires_grad=False, device=device, dtype=dtype)
pi = torch.tensor(data.delay_distr, requires_grad=False, device=device, dtype=dtype)
serial_interval = torch.tensor(data.serial_interval, requires_grad=False, device=device, dtype=dtype)
population = torch.tensor(5793636, requires_grad=False, device=device, dtype=dtype)
num_observations = len(observed_daily_hospit)
model = [observed_daily_hospit,pi,serial_interval,num_observations]

Initialize latent variables / parameters

1. initialize (constrained )variables
2. transform to unconstrained
3.1 transform unconstrained to constrained, xis the unconstrained -> theta constrained  theta = f(x)
3.2 loss = evaluate(model, x)   evaluate(model, f(x))
3.3 gradient(loss)   #loss.backward()#gradient(  x -> evaluate(model, f(x)), x )

3.4 optimize (e.g. SGD: x -= learningrate * x.grad)
3.5 return to 3.1



In [5]:
tau = np.random.exponential(1 / 0.03)
tau_t = torch.tensor(tau, requires_grad=True, device=device, dtype=dtype)
# b=(upper-lower)/scale, loc=lower, scale=scale
y = torch.tensor(truncexpon.rvs(b=(1000 - 0) / tau, loc=0, scale=tau), requires_grad=False, device=device, dtype=dtype)  # number of initial newly_infected (seed)

# For trunc ((lower - mu) / sigma, (upper - mu) / sigma, loc=mu, scale=sigma)
phi = torch.tensor(truncnorm.rvs((0 - 25) / 10, (np.inf - 25) / 10, loc=25, scale=10), requires_grad=True, device=device, dtype=dtype)  # dispersion (shape) parameter for observations
R0 = torch.tensor(truncnorm.rvs((2 - 3.6) / 0.8, (5 - 3.6) / 0.8, loc=3.6, scale=0.8), requires_grad=True, device=device, dtype=dtype)  # initial reproduction number
alpha = torch.tensor(truncnorm.rvs((0 - 1/100) / 1/100, (5/100 - 1/100) / 1/100, loc=1/100, scale=1/100), requires_grad=True, device=device, dtype=dtype)  # probability to get hospitalized
sigma = torch.tensor(truncnorm.rvs((0 - 0.1) / 0.3, (0.5 - 0.1) / 0.3, loc=0.1, scale=0.3), requires_grad=True, device=device, dtype=dtype)  # standart deviation of random walk step

__From Andreas' model__

"""
(model::Covid19Model)(θ) returns the log loglikelihood of our model with
respect to the parameters θ together with inferred values of 'Rt' and
'expected_daily_hospit'.

In a 'standard' machine learning model the loss function is something like
loss(y,ŷ,θ) = sum( (y-ŷ)^2 ) + α*θ,
where 'y' is the observed outcome, 'ŷ' is the model estimate and α is some
regularisation parameter.

In a Bayesian framework the first term is called the log loglikelihood of the
data wrt. the model and the regularisation term is the log likelihood of our
parameters θ wrt. the prior.

The 'standard' loss function is a special case of the Bayesian approach which
assumes Gaussian distribtions.
"""

function evaluate(model::Covid19Model, θ)
	@unpack τ, y, ϕ, R0, α, σ, ϵt = θ
	@unpack num_obs, num_impute, observed_hospit, i2h, pop, si = model
    ℓ = 0. #likelihood

	# log likelihood wrt. our prior ("regularisation")
	ℓ += logpdf( Exponential(1 / 0.03), τ)
	T = typeof(τ)
	ℓ += logpdf( truncated(Exponential(τ), T(0), T(1000)), y) # number of initial newly_infected (seed)
	ℓ += logpdf( truncated(Normal(25, 5), 0, Inf), ϕ) # dispersion (shape) parameter for observations
	ℓ += logpdf( truncated(Normal(3.6, 0.8), 2, 5), R0) # initial reproduction number
	ℓ += logpdf( truncated(Normal(1/100,1/100), 0,5/100), α) # probability to get hospitalized
	ℓ += logpdf( truncated(Normal(0.05, 0.03), 0, 0.15), σ) # standart deviation of random walk step

# Define Forward Pass

In [17]:
def forward_pass():
  # Initialize time series variables
  newly_infected = torch.zeros(num_observations)  # number of newly infected
  effectively_infectious = torch.zeros(num_observations)  # effective number of infectious individuals
  expected_daily_hospit = torch.zeros(num_observations)  # expected number of daily hospitalizations
  cumulative_infected = torch.zeros(num_observations)  # cumulative number of infected
  eta_t = torch.zeros(num_observations)  # transformed reproduction number
  epsilon_t = torch.zeros(num_observations)  # random walk
  St = torch.zeros(num_observations)  # fraction of susceptible population

  # log likelihood wrt. our prior ("regularisation")
  # ll stands for log-likelihood
  ll = torch.tensor(0)
  ll += torch.tensor(expon.logpdf(tau_t, 1 / 0.03))
  ll += torch.tensor(truncexpon.logpdf(y,b=(1000 - 0) / tau_t, loc=0, scale=tau_t))
  ll += torch.tensor(truncnorm.logpdf(phi,(0 - 25) / 10, (np.inf - 25) / 10, loc=25, scale=10))
  ll += torch.tensor(truncnorm.logpdf(R0,(2 - 3.6) / 0.8, (5 - 3.6) / 0.8, loc=3.6, scale=0.8))
  ll += torch.tensor(truncnorm.logpdf(alpha,(0 - 1/100) / 1/100, (5/100 - 1/100) / 1/100, loc=1/100, scale=1/100))
  ll += torch.tensor(truncnorm.logpdf(sigma,(0 - 0.1) / 0.3, (0.5 - 0.1) / 0.3, loc=0.1, scale=0.3))

  # seed initial infection / impute first num_impute days
  newly_infected[0:num_impute] = y.clone()
  cumulative_infected[0] = 0.
  cumulative_infected[1:num_impute] = torch.cumsum(newly_infected[0:num_impute - 1].clone(), dim=0)
  St[0:num_impute] = torch.tensor([torch.maximum(population.clone() - x, torch.tensor(0)) / population for x in cumulative_infected[0:num_impute].clone()])

  # calculate Rt: the basic reproduction number
  # basic reproduction number as a latent random walk
  beta_0 = torch.log(R0)
  #epsilon_t[0] = torch.distributions.Normal(cero, sigma).rsample()
  #for t in range(1, num_observations):
  #    epsilon_t[t] = torch.distributions.Normal(epsilon_t[t - 1].clone(), sigma).rsample()
  #eta_t = beta_0 + epsilon_t  # + RNN[X_t, t]  # .clone() necessary?
  eta_t[0] = beta_0
  epsilon_t[0] = torch.distributions.Normal(cero, sigma).rsample()
  for t in range(1, num_observations):
      epsilon_t[t] = torch.distributions.Normal(epsilon_t[t - 1].clone(), sigma).rsample()
      dist_epsilon_t = torch.distributions.Normal(epsilon_t[t - 1], sigma)
      ll += dist_epsilon_t.log_prob(epsilon_t[t - 1]) #epsilon_t.log_prob(epsilon_t[t - 1])
      #eta_t[t] = epsilon_t[t-1]  
  eta_t[1:num_observations] = beta_0 + epsilon_t[0:num_observations-1].clone() 
  Rt = torch.exp(eta_t)

  # calculate infections
  for t in range(num_impute, num_observations):
      # Update cumulative newly_infected
      cumulative_infected[t] = cumulative_infected[t - 1].clone() + newly_infected[t - 1].clone()
      # Adjusts for portion of pop that are susceptible
      St[t] = torch.maximum(population.clone() - cumulative_infected[t].clone(), cero) / population.clone()
      # effective number of infectous individuals
      for i in range(0, t - 1):
          effectively_infectious[t] += newly_infected[i].clone() * serial_interval[t - i].clone()
      newly_infected[t] = St[t].clone() * Rt[t].clone() * effectively_infectious[t].clone()

  # calculate expected number of hospitalizations
  expected_daily_hospit[0] = (1e-15) * newly_infected[0].clone()
  for t in range(1, num_observations):
      for i in range(0, t - 1):
          expected_daily_hospit[t] += newly_infected[i].clone() * pi[t - i].clone()
  expected_daily_hospit = alpha * expected_daily_hospit

	# compare observed hospitalizations to model results
	# likelihood of the data wrt. to the model 
  for i in range(1, num_observations):
      dist = torch.distributions.negative_binomial.NegativeBinomial(phi, 1/(1+ expected_daily_hospit[i]/phi))
      ll += dist.log_prob(observed_daily_hospit[i])


  return expected_daily_hospit, Rt, ll

```
	# basic reproduction number as a latent random walk
	β0 = log(R0)
	ηt[1] = β0
	for t in 2:num_obs
		ℓ += logpdf( Normal(ηt[t-1], σ), ϵt[t-1]) # log likelihood wrt. our prior ("regularisation")
		ηt[t] = ϵt[t-1] # + RNN[X_t, t]
	end
	Rt = exp.(ηt)
```

In [27]:
forward_pass()

RuntimeError: ignored

# Optimization

In [18]:
learning_rate = 1e-12
for t2 in range(10):

  for t in range (40):
    # forward pass - calculate expected_daily_hospit
    expected_daily_hospit, Rt, ll, theta = forward_pass()

    loss = -ll

    loss.backward()

    print(f'Time Step: {t}, Loss: {loss}, Observed_daily_hospit: {torch.sum(observed_daily_hospit)}, Expected_daily_hospit: {torch.sum(expected_daily_hospit)}')
    

    with torch.no_grad(): # this part is SGD. can also replace with loss.step
      #tau_t -= learning_rate * tau_t.grad
      #phi -= learning_rate * phi.grad
      R0 -= learning_rate * R0.grad
      alpha -= learning_rate * alpha.grad
      sigma -= learning_rate * sigma.grad
      print(f' R0:{R0}, grad: {R0.grad}, alpha: {alpha} grad: {alpha.grad}, sigma: {sigma} grad {sigma.grad}' )

      #tau_t.grad = None
      #phi.grad = None
      R0.grad = None
      alpha.grad = None
      sigma.grad = None
  
  
  plt.plot(expected_daily_hospit.detach().numpy(), label='expected_daily_hospit')
  plt.plot(observed_daily_hospit.detach().numpy(), label='observed_daily_hospit')
  plt.legend()
  plt.show()

RuntimeError: ignored

compare observed hospitalizations to model results

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(expected_daily_hospit.detach().numpy(), label='expected_daily_hospit')
plt.plot(observed_daily_hospit.detach().numpy(), label='observed_daily_hospit')
plt.legend()
plt.show()